# ООП. Часть 2

## Статические методы


для функций, которые не относятся к конкретным экземплярам класса

- можем использовать для логического объединения функций
- не принимает в себя self! (ссылку на экземпляр класса)
- искользуются как вспомогательные функции, которые работают с данными, которые в них передают в качестве агрументов
- не могут влиять на состояние класса или экземпляра

In [12]:
class Person:
    
    @staticmethod
    def is_adult(age):
        return age >= 18


In [13]:
age = 17
print(Person.is_adult(age))

False


## Методы класса

- принимают в себя класс cls
- метод привязан к классу, а не к экземпляру
- могут менять состояние класса, а это в свою очередь может отразиться на всех объектах, но не могут менять конкретный объект

In [3]:
from datetime import date


class Person:
    
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    @classmethod
    def from_birth_year(cls, name, year):
        return cls(name, date.today().year - year)
    
    @staticmethod
    def is_adult(age):
        return age > 18
    
    def say(self):
        print("Hi")
    
person1 = Person('Sarah', 25)
person2 = Person.from_birth_year('Roark', 1994)

In [4]:
person2.say()

Hi


In [2]:
person2.age

1994

### Паттерн singleton

In [5]:
class Singleton:
    def __new__(cls): # new управляет созданием нового экземпляра класса
        if not hasattr(cls, 'instance'):
            cls.instance = super(Singleton, cls).__new__(cls)
        return cls.instance

singleton = Singleton()
new_singleton = Singleton()

<class 'type'>
<class 'type'>


In [22]:
singleton = Singleton()

In [23]:
new_singleton = Singleton()

In [24]:
singleton == new_singleton

True

In [11]:
class Teacher:
    def __new__(cls): # new управляет созданием нового экземпляра класса
        if not hasattr(cls, 'teacher'):
            cls.teacher = super(Teacher, cls).__new__(cls)
        return cls.teacher

t_1 = Teacher()
t_2 = Teacher()

In [12]:
id(t_1)

2413971031232

In [13]:
id(t_2)

2413971031232

In [ ]:
a = 2
a = 3

In [14]:
class Animal:
    allowed_types = ['cat', 'dog']
    
    def __init__(self, name):
        self.name = name

In [15]:
Animal.allowed_types

['cat', 'dog']

## Property

когда хотим управлять доступом к атрибутам класса

то есть обеспечивает интерфейс для атрибуетов экземпляра класса

In [16]:
class User:
    def __init__(self, name, password):
        self.name = name
        self.password = password # тут вызываем password.setter
        
    @property
    def password(self): # getter
        return f"{self.__password[:2]}...{self.__password[-2:]}"
    
    @password.setter
    def password(self, password): # setter
        if len(password) < 8: # тут может быть логика по верификации
            raise Exception("Недостаточная длина пароля.")
        self.__password = password
    

In [17]:
user = User(name="Lena", password="1234")

Exception: Недостаточная длина пароля.

In [18]:
user = User(name="Lena", password="12345678")

In [19]:
user.name

'Lena'

In [20]:
user.password

'12...78'

In [40]:
user._User__password

b'12345678'

Задание: когда используем classmethod, а когда staticmethod?

Задание: написать класс Animal, у которого есть name, animal_type и voice (то что говорит животное) - animal_type может быть только dog или cat, voice по дефолту None

In [26]:
class Animal:
    
    animal_types = ['cat', 'dog']
    
    def __init__(self, name, animal_type, voice=None):
        self.name = name
        self.animal_type = animal_type
        self.voice = voice
        
    @property
    def animal_type(self):
        return self._animal_type
    
    @animal_type.setter
    def animal_type(self, animal_type):
        if animal_type in Animal.animal_types:
            self._animal_type = animal_type
        else:
            raise Exception("Неверный тип животного.")

animal = Animal(name="Кот", animal_type="cat")
animal.animal_type

'cat'

In [25]:
Animal.animal_types

['cat', 'dog']

добавить в этот класс метод, который умеет создавать животное с нужным voice, основываясь на его типе

In [52]:
class Animal:
    
    animal_types = ['cat', 'dog']
    
#     def __new__(cls): # new управляет созданием нового экземпляра класса
#         if not hasattr(cls, 'instance'):
#             cls.instance = super(Animal, cls).__new__(cls)
#         return cls.instance
    
    def __init__(self, name, animal_type, voice=None):
        self.name = name
        self.animal_type = animal_type
        self.voice = voice
        
    @property
    def animal_type(self):
        return self._animal_type
    
    @animal_type.setter
    def animal_type(self, animal_type):
        if animal_type in Animal.animal_types:
            self._animal_type = animal_type
        else:
            raise Exception("Неверный тип животного.")
    
    @classmethod      
    def from_animal_type(cls, name, animal_type):
        voice = "мяу" if animal_type == "cat" else "гав"
        return cls(name, animal_type, voice)
    
    def make_sound(self):
        print(f"{self.animal_type} говорит {self.voice}")

animal = Animal.from_animal_type("Маркиза", "cat")
animal.name

'Маркиза'

In [44]:
print(animal.voice)

мяу


In [45]:
animal.make_sound()

cat говорит мяу


In [46]:
animal = Animal("Маркиза", "cat", "mrrr")
animal.make_sound()

cat говорит mrrr


In [47]:
animal.voice = "гав-гав" # переопределить атрибут кота - voice
animal.make_sound()

cat говорит гав-гав


сделать так чтобы мы могли создать только 1 животное с определенным типом (одного кота, одну собачку)

In [48]:
class Cat:
    def __new__(cls): # new управляет созданием нового экземпляра класса
        if not hasattr(cls, 'cat'):
            cls.cat = super(Cat, cls).__new__(cls)
        return cls.cat

cat_1 = Cat()
cat_2 = Cat()
cat_3 = cat_1 # всегда равны, потому что это один и тот же объект

In [49]:
id(cat_1)

2413976656320

In [50]:
id(cat_2)

2413976656320

In [51]:
class Dog:
    def __new__(cls): # new управляет созданием нового экземпляра класса
        if not hasattr(cls, 'dog'):
            cls.dog = super(Dog, cls).__new__(cls)
        return cls.dog

dog_1 = Dog()
dog_2 = Dog()
dog_3 = dog_1 # всегда равны, потому что это один и тот же объект

написать метод, который выводит на экран список допустимых животных

In [53]:
class Animal:
    
    animal_types = ['cat', 'dog']
    
    def __init__(self, name, animal_type, voice=None):
        self.name = name
        self.animal_type = animal_type
        self.voice = voice
        
    @staticmethod
    def print_animal_types():
        print(Animal.animal_types)

In [54]:
Animal.print_animal_types()

['cat', 'dog']


## Метаклассы - фабрики классов

Зачем?
- можно динамически создавать классы, то есть автоматически изменять классы во время создания (например, мы хотим чтобы все классы в модуле или атрибуты в верхнем регистре)
- возвращают классы
- часто используются для реализации API типа Django
- редко используется в обычной практике
- могут создаваться при помощи функции type


[статья](https://habr.com/ru/articles/145835/)

In [41]:
MyDymanicClass = type("MyDymanicClass", (), {})

In [43]:
type(MyDymanicClass)

type

In [44]:
MyDymanicClass = type("MyDymanicClass", (), {"attribute_example": "example"})

In [45]:
my_dynamic_instance = MyDymanicClass()

In [46]:
type(my_dynamic_instance)

__main__.MyDymanicClass

In [47]:
my_dynamic_instance.attribute_example

'example'

Как это делается в реальной жизни

In [ ]:
class Foo:
    __metaclass__ = something...    
    [...]

## Дескрипторы 

[статья](https://pythonist.ru/deskriptory-v-python/)

[руководство по дескрипторам](https://habr.com/ru/articles/122082/)

## MRO - method resolution order

порядок разрешения методов (и атрибутов)

[порядок разрешения методов в python](https://habr.com/ru/articles/62203/)

[и еще одна, возможно более понятная](https://tirinox.ru/mro-python/)

In [65]:
class Employee:    
    @property
    def responsibilities(self):
        return ["ходить в офис", "предоставлять отчеты", "работать с клиентами"]
    
    
class Parent:
    @property
    def responsibilities(self):
        return ["кормить ребенка", "следить за здоровьем", "играть", "развивать"]
    

class Person(Parent, Employee): # какой тут приоритет? тут человек в первую очередь родитель
    pass

In [66]:
person = Person()

In [67]:
person.responsibilities

['кормить ребенка', 'следить за здоровьем', 'играть', 'развивать']

In [61]:
Person.__mro__

(__main__.Person, __main__.Parent, __main__.Employee, object)

In [56]:
class Employee:    
    @property
    def responsibilities(self):
        return ["ходить в офис", "предоставлять отчеты", "работать с клиентами"]
    
    
class Parent:
    @property
    def responsibilities(self):
        return ["кормить ребенка", "следить за здоровьем", "играть", "развивать"]
    
    @property
    def featues(self):
        return ["есть маленький человек, который похож на тебя"]
    

class Person(Employee, Parent):
    pass

In [57]:
person = Person()
person.featues

['есть маленький человек, который похож на тебя']

In [68]:
person.method()

AttributeError: 'Person' object has no attribute 'method'

Задание: как явно вызвать родительский метод?

In [73]:
class Employee: 
    def say_hi(self):
        print("hi from Employee")
    
class Parent:
    @property
    def responsibilities(self):
        return ["кормить ребенка", "следить за здоровьем", "играть", "развивать"]
    
    def say_hi(self):
        print("hi from Parent")
    

class Person(Employee, Parent):    
    
    def say_hi(self):
        Parent.say_hi(self)
        Employee.say_hi(self)
        
person = Person()
person.say_hi()

hi from Parent
hi from Employee


## Mixin - классы-примеси

[пример](https://www.codecamp.ru/blog/python-mixins/)

- предоставить множество дополнительных функций для класса;
- использовать определённую функцию во множестве разных классов.

In [12]:
class YandexMusicMixin:
    
    def play_yandex_music(self):
        print("сейчас играет подборка дня")
        
        
class Station:
    def __init__(self):
        self.is_off = True
    
    def turn_on(self):
        self.is_off = False
        
    def turn_off(self):
        self.is_off = True
    
    
class YandexStation(Station, YandexMusicMixin):
    pass


class Phone:
    def __init__(self, model):
        self.model = model
        
        
class GooglePixel(Phone, YandexMusicMixin):
    pass

In [13]:
yandex_station = YandexStation()
yandex_station.play_yandex_music()

сейчас играет подборка дня


In [14]:
yandex_station.turn_off()

In [15]:
yandex_station.is_off

True

In [16]:
pixel = GooglePixel("pixel 6")
pixel.model

'pixel 6'

In [17]:
pixel.play_yandex_music()

сейчас играет подборка дня


## Абстрактные классы / методы

[ссылка](http://pythonicway.com/education/python-oop-themes/33-python-abstract-class)

- нужны для того чтобы определить интерфейс и сказать что все дочерние классы должны его реализовать
- нельзя создать экземпляр абстрактного класса
- нельзя создать экземпляр класса с нереализованными абстрактными методами

In [93]:
from abc import ABC, abstractmethod

class Animal(ABC):
    def __init__(self, name):
        self.name = name
    
    @abstractmethod
    def make_sound(self):
        pass

In [94]:
class Cat(Animal):
    
     def make_sound(self):
        print("meow")
        
cat = Cat("Kacпер")
cat.make_sound()
cat.name

meow


'Kacпер'

In [95]:
animal = Animal("random animal")

TypeError: Can't instantiate abstract class Animal with abstract method make_sound

In [4]:
class Dog(Animal):
    pass

dog = Dog("Кексик")

TypeError: Can't instantiate abstract class Dog with abstract method make_sound

Можно самостоятельно реализовать:

In [86]:
class Animal:
    def __init__(self, name):
        self.name = name
        
    def say_hi(self):
        print("hi")
    
    def make_sound(self):
        raise NotImplementedError("метод make_sound должен быть реализован в дочерних классах!")
        
class Dog(Animal):
    pass

dog = Dog("Кексик")

In [87]:
dog.make_sound()

NotImplementedError: метод make_sound должен быть реализован в дочерних классах!

In [88]:
class Dog(Animal):
    def make_sound(self):
        print("гав")
        
dog = Dog("Кексик")
dog.make_sound()

гав


In [89]:
dog.say_hi()

hi


## Менеджер контекста - with


- можно написать свой, для этого нужно реализовать методы __ enter __ , __ exit __ 
- нужны для того чтобы выделять и освобождать ресурсы строго по необходимости (и не забыть что-то открыть или закрыть)

[почитать](https://pavel-karateev.gitbook.io/intermediate-python/sintaksis/context_managers)

In [10]:
class File:
    def __init__(self, file_name, method):
        self.file_obj = open(file_name, method)
        
    def __enter__(self):
        return self.file_obj
        
    def __exit__(self, type, value, traceback):
        self.file_obj.close()

In [11]:
with File('demo.txt', 'w') as opened_file:
    opened_file.write('Hola!')
    
print()

In [96]:
with open('demo.txt', 'w') as opened_file:
    opened_file.write('Hola!')

# Домашнее задание

#### Техническое задание:

Есть магазин, продукт (товар), покупатель, корзина и список покупок - это все отдельные классы, атрибуты и методы которых нужно продумать самостоятельно для того чтобы реализовать логику ниже


#### Часть 1. Спроектировать классы.

Написать классы:

- Продукт - атрибуты: id, name, price, amount
- Магазин - список продуктов (объектов класса Product)
- Покупатель - id, количество денег, список продуктов (названия: молоко, сыр и тд), которые нужно купить - все задается при инициализации покупателя
- Корзина - id, список продуктов (объектов класса Product - по умолчанию None)

#### Часть 2. Написать бизнес-логику

Реализовать следующие возможности:

1. Создается покупатель с определенным количеством денег и списком продуктов
2. Покупатель приходит в магазин и берет корзину (покупателю при помощи метода создается и добавляется корзина)
3. Создать экземпляр магазина, который должен быть синглтоном, в котором будет список продуктов
4. Покупатель проходится по списку продуктов, которые ему нужно купить и смотрит, есть ли в магазине эти продукты, если продукт есть, то покупатель добавляет его в свою корзину (нужен метод для добавления продукта Product в корзину) ТОЛЬКО ПРИ УСЛОВИИ что у покупателя достаточно денег. В ином случае вывести сообщение. Если денег достаточно, то покупатель добавляет продукт в корзину и количество этого продукта в магазине уменьшается на 1
5. После того как прошлись по всем продуктам, вывести сообщение о том, какие продукты были куплены, а какие не были



Четкого ТЗ нет, фантазия приветствуется!

#### Часть 3* 

Добавить магазину дополнительные возможности с использованием абстрактных, статических и методов класса